In [ ]:
!pip install torch torchvision transformers datasets accelerate scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import os
import json
import torch
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from collections import Counter

from transformers import (
    ViTForImageClassification,
    ViTImageProcessor,
    Trainer,
    TrainingArguments,
    EvalPrediction
)

In [ ]:
from transformers.trainer_utils import get_last_checkpoint
from google.colab import drive

from torchvision.datasets import ImageFolder
from torchvision import transforms
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import WeightedRandomSampler

In [ ]:
# ✅ Mount Drive
drive.mount('/content/drive')

# ✅ Dataset paths
BASE_PATH = '/content/drive/MyDrive/MultiBanFake/Dataset'
TRAIN_DIR = os.path.join(BASE_PATH, 'Train')
VAL_DIR   = os.path.join(BASE_PATH, 'Validation')
TEST_DIR  = os.path.join(BASE_PATH, 'Test')


Mounted at /content/drive


In [ ]:
# ✅ Load processor
processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

In [ ]:
# ✅ Define transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std),
])

eval_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std),
])

In [ ]:
# ✅ Load datasets
train_dataset = ImageFolder(root=TRAIN_DIR, transform=train_transform)
val_dataset   = ImageFolder(root=VAL_DIR, transform=eval_transform)
test_dataset  = ImageFolder(root=TEST_DIR, transform=eval_transform)

label_names = train_dataset.classes
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {label: i for i, label in enumerate(label_names)}

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        return {"pixel_values": image, "labels": label}

train_ds = CustomDataset(train_dataset)
val_ds   = CustomDataset(val_dataset)
test_ds  = CustomDataset(test_dataset)


In [ ]:
# ✅ Class weights and sampler
train_targets = train_dataset.targets
class_weights = compute_class_weight('balanced', classes=np.unique(train_targets), y=train_targets)
sample_weights = [class_weights[t] for t in train_targets]
sampler = WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)

# ✅ Metrics
def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    report = classification_report(labels, preds, target_names=label_names, output_dict=True)
    print(json.dumps(report, indent=2))
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": report['weighted avg']['f1-score'],
    }

In [ ]:
# ✅ Load model
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=len(label_names),
    id2label=id2label,
    label2id=label2id,
)
model.gradient_checkpointing_enable()

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# ✅ Training Arguments (compatible)
training_args = TrainingArguments(
    output_dir="./vit_mresults",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-5,
    warmup_steps=500,
    weight_decay=0.01,
    save_strategy="epoch",
    save_total_limit=1,
    logging_dir="./logs",
    logging_steps=10,
    fp16=True,
    do_train=True,
    do_eval=True,
    label_smoothing_factor=0.1
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=processor,
    compute_metrics=compute_metrics,
    data_collator=None
)


/tmp/ipython-input-15-831303331.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# ✅ Train with checkpoint support
checkpoint = get_last_checkpoint("./vit_mresults")
if checkpoint:
    print(f"Resuming from checkpoint: {checkpoint}")
    trainer.train(resume_from_checkpoint=checkpoint)
else:
    print("Starting training from scratch...")
    trainer.train()

Starting training from scratch...


Step,Training Loss
10,0.692600
20,0.687100
30,0.686300
40,0.690500
50,0.693000
60,0.682900
70,0.694700
80,0.691200
90,0.703200
100,0.699400


Step,Training Loss
10,0.692600
20,0.687100
30,0.686300
40,0.690500
50,0.693000
60,0.682900
70,0.694700
80,0.691200
90,0.703200
100,0.699400


In [ ]:
# ✅ Save final model
model.save_pretrained("/content/vit_final_model")

In [ ]:
# ✅ Evaluate and save results
preds = trainer.predict(test_ds)
pred_labels = np.argmax(preds.predictions, axis=1)
true_labels = preds.label_ids

pred_counts = dict(Counter(pred_labels))
print("\n🔍 Predicted Class Distribution:", pred_counts)

report = classification_report(true_labels, pred_labels, target_names=label_names, digits=4)
print("\n📊 Fake vs Non-Fake Class Report:\n", report)

with open("vit_classification_report.json", "w") as f:
    json.dump(report, f, indent=4)

probs = torch.nn.functional.softmax(torch.tensor(preds.predictions), dim=-1).numpy().tolist()

with open("vit_stack_probs.json", "w") as f:
    json.dump({
        "vit_probs": probs,
        "true_labels": true_labels.tolist()
    }, f)

print("\n✅ Saved model, classification report, and stacking probabilities.")

{
  "Fake": {
    "precision": 0.6676056338028169,
    "recall": 0.4968553459119497,
    "f1-score": 0.5697115384615384,
    "support": 477.0
  },
  "Real": {
    "precision": 0.6013289036544851,
    "recall": 0.7541666666666667,
    "f1-score": 0.6691312384473198,
    "support": 480.0
  },
  "accuracy": 0.625914315569488,
  "macro avg": {
    "precision": 0.634467268728651,
    "recall": 0.6255110062893081,
    "f1-score": 0.6194213884544291,
    "support": 957.0
  },
  "weighted avg": {
    "precision": 0.634363386706475,
    "recall": 0.625914315569488,
    "f1-score": 0.6195772187051904,
    "support": 957.0
  }
}

🔍 Predicted Class Distribution: {np.int64(1): 602, np.int64(0): 355}

📊 Fake vs Non-Fake Class Report:
               precision    recall  f1-score   support

        Fake     0.6676    0.4969    0.5697       477
        Real     0.6013    0.7542    0.6691       480

    accuracy                         0.6259       957
   macro avg     0.6345    0.6255    0.6194       9